In [ ]:
%pip install -q numpy pandas scipy scikit-learn torch tqdm gensim matplotlib mol2vec esm
%pip install -q rdkit psutil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 105.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 126.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 

In [ ]:
!python -c "from esm.models.esmc import ESMC; print('ESM-C installed successfully!')"

ESM-C installed successfully!


In [ ]:
import os
import pathlib
import subprocess

!git clone --branch new-prot-embs https://github.com/glucose20/Temp.git

Cloning into 'Temp'...
remote: Enumerating objects: 215, done.
remote: Counting objects: 100% (170/170), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 215 (delta 69), reused 147 (delta 52), pack-reused 45 (from 2)
Receiving objects: 100% (215/215), 94.48 MiB | 21.09 MiB/s, done.
Resolving deltas: 100% (70/70), done.


In [ ]:
# Import library
import kagglehub

# Download dataset from Kaggle (LLMDTA dataset)
path = kagglehub.dataset_download("christang0002/llmdta")

!ls {path}/pretrain-feature/pretrained-feature

100%|██████████| 2.90G/2.90G [01:16<00:00, 40.8MB/s]

Extracting files...


davis  kiba  metz


In [ ]:
import os
import shutil


source_dir1 = f'{path}/pretrain-feature/pretrained-feature/davis'
source_dir2 = f'{path}/pretrain-feature/pretrained-feature/kiba'
source_dir3 = f'{path}/pretrain-feature/pretrained-feature/metz'
destination_dir = '/content/Temp/data'

# Remove destination directories if they exist
if os.path.exists(os.path.join(destination_dir, os.path.basename(source_dir1))):
    shutil.rmtree(os.path.join(destination_dir, os.path.basename(source_dir1)))
if os.path.exists(os.path.join(destination_dir, os.path.basename(source_dir2))):
    shutil.rmtree(os.path.join(destination_dir, os.path.basename(source_dir2)))
if os.path.exists(os.path.join(destination_dir, os.path.basename(source_dir3))):
    shutil.rmtree(os.path.join(destination_dir, os.path.basename(source_dir3)))

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Copy the directory
shutil.copytree(source_dir1, os.path.join(destination_dir, os.path.basename(source_dir1)))
shutil.copytree(source_dir2, os.path.join(destination_dir, os.path.basename(source_dir2)))
shutil.copytree(source_dir3, os.path.join(destination_dir, os.path.basename(source_dir3)))

print(f"Copied {source_dir3} to {destination_dir}")

Copied /root/.cache/kagglehub/datasets/christang0002/llmdta/versions/3/pretrain-feature/pretrained-feature/metz to /content/Temp/data


In [ ]:
!tar -xzf /content/Temp/data/dta-5fold-dataset/davis.tar.gz -C /content/Temp/data/dta-5fold-dataset/
!tar -xzf /content/Temp/data/dta-5fold-dataset/kiba.tar.gz -C /content/Temp/data/dta-5fold-dataset/
!tar -xzf /content/Temp/data/dta-5fold-dataset/metz.tar.gz -C /content/Temp/data/dta-5fold-dataset/

In [ ]:
%cd /content/Temp

/content/Temp


In [ ]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle

# ESM-C imports
from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, LogitsConfig

In [ ]:
# Select ESM-C model variant
MODEL_NAME = "esmc_600m"  # Options: esmc_300m, esmc_600m, esmc_6b

# Device configuration
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# Model dimensions
MODEL_DIMS = {
    "esmc_300m": 960,
    "esmc_600m": 1152,
    "esmc_6b": 2560
}
EMBEDDING_DIM = MODEL_DIMS[MODEL_NAME]
print(f"Model: {MODEL_NAME}, Embedding dimension: {EMBEDDING_DIM}")

Using device: cuda
Model: esmc_600m, Embedding dimension: 1152


In [ ]:
# Load model
print(f"Loading {MODEL_NAME}...")
model = ESMC.from_pretrained(MODEL_NAME).to(DEVICE)
model.eval()  # Set to evaluation mode
print("Model loaded successfully!")

Loading esmc_600m...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

.gitattributes: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

data/weights/esmc_600m_2024_12_v0.pth:   0%|          | 0.00/2.30G [00:00<?, ?B/s]

Model loaded successfully!


In [ ]:
def get_esmc_pretrain(model, df_dir, db_name, sep=' ', header=None,
                      col_names=['drug_id', 'prot_id', 'drug_smile', 'prot_seq', 'label']):
    """
    Extract ESM-C embeddings for protein sequences.

    Args:
        model: Loaded ESM-C model
        df_dir: Path to input CSV file
        db_name: Database name for output filename
        sep: CSV separator
        header: CSV header row (None for no header)
        col_names: Column names for the dataframe

    Returns:
        Dictionary containing embeddings
    """
    # Load data
    df = pd.read_csv(df_dir, sep=sep, header=header)
    df.columns = col_names
    df.drop_duplicates(subset='prot_id', inplace=True)

    prot_ids = df['prot_id'].tolist()
    prot_seqs = df['prot_seq'].tolist()

    emb_dict = {}
    emb_mat_dict = {}
    length_dict = {}

    print(f"Processing {len(prot_ids)} proteins...")

    for idx in tqdm(range(len(prot_ids))):
        prot_id = str(prot_ids[idx])
        # ESM-C supports up to 2048 tokens (vs ESM2's 1022)
        seq = prot_seqs[idx][:2048]
        length_dict[prot_id] = len(seq)

        try:
            # Create ESMProtein object
            protein = ESMProtein(sequence=seq)

            # Encode the protein
            protein_tensor = model.encode(protein)

            # Extract embeddings
            with torch.no_grad():
                logits_output = model.logits(
                    protein_tensor,
                    LogitsConfig(return_embeddings=True)
                )

            # Get embeddings (shape: seq_len x d_model)
            embeddings = logits_output.embeddings.cpu().numpy()

            # Store mean embedding as sequence representation
            emb_dict[prot_id] = embeddings.mean(axis=0)  # Shape: (d_model,)

            # Store full embedding matrix
            emb_mat_dict[prot_id] = embeddings  # Shape: (seq_len, d_model)

        except Exception as e:
            print(f"\nError processing {prot_id}: {e}")
            # Use zero embeddings as fallback
            emb_dict[prot_id] = np.zeros(EMBEDDING_DIM)
            emb_mat_dict[prot_id] = np.zeros((len(seq), EMBEDDING_DIM))

    # Prepare output dictionary
    dump_data = {
        "dataset": db_name,
        "vec_dict": emb_dict,
        "mat_dict": emb_mat_dict,
        "length_dict": length_dict,
        "model": MODEL_NAME,
        "embedding_dim": EMBEDDING_DIM
    }

    # Save to pickle
    output_file = f'./{db_name}_esmc_600m_pretrain.pkl'
    with open(output_file, 'wb') as f:
        pickle.dump(dump_data, f)

    print(f"\nSaved embeddings to: {output_file}")
    print(f"Total proteins: {len(emb_dict)}")
    print(f"Embedding dimension: {EMBEDDING_DIM}")

    return dump_data

In [ ]:
# Example: Process case study data
db_name = 'case_study'
df_dir = '/content/Temp/data/simple-Case/proteins.csv'  # Update this path
col_names = ['prot_id', 'prot_seq']

# Uncomment to run:
result = get_esmc_pretrain(model, df_dir, db_name, sep=',', header=0, col_names=col_names)

Processing 4 proteins...


100%|██████████| 4/4 [00:08<00:00,  2.18s/it]


Saved embeddings to: ./case_study_esmc_600m_pretrain.pkl
Total proteins: 4
Embedding dimension: 1152


In [ ]:
# Dataset configurations
datasets = {
    'davis': './data/dta-5fold-dataset/davis/davis_prots.csv',
    'kiba': './data/dta-5fold-dataset/kiba/kiba_prots.csv',
    'metz': './data/dta-5fold-dataset/metz/metz_prots.csv'
}

col_names = ['prot_id', 'prot_seq']

# Process each dataset
for db_name, df_dir in datasets.items():
    print(f"\n{'='*60}")
    print(f"Processing {db_name.upper()} dataset")
    print(f"{'='*60}")

    try:
        result = get_esmc_pretrain(
            model=model,
            df_dir=df_dir,
            db_name=db_name,
            sep=',',  # Adjust if needed
            header=0,
            col_names=col_names
        )
        print(f"Successfully processed {db_name}")
    except Exception as e:
        print(f"Error processing {db_name}: {e}")


Processing DAVIS dataset
Processing 442 proteins...


100%|██████████| 442/442 [03:46<00:00,  1.95it/s]



Saved embeddings to: ./davis_esmc_600m_pretrain.pkl
Total proteins: 442
Embedding dimension: 1152
Successfully processed davis

Processing KIBA dataset
Processing 229 proteins...


100%|██████████| 229/229 [01:48<00:00,  2.12it/s]



Saved embeddings to: ./kiba_esmc_600m_pretrain.pkl
Total proteins: 229
Embedding dimension: 1152
Successfully processed kiba

Processing METZ dataset
Processing 170 proteins...


100%|██████████| 170/170 [01:19<00:00,  2.15it/s]



Saved embeddings to: ./metz_esmc_600m_pretrain.pkl
Total proteins: 170
Embedding dimension: 1152
Successfully processed metz


In [ ]:
# Load and inspect generated embeddings
def verify_embeddings(db_name):
    file_path = f'./{db_name}_esmc_pretrain.pkl'

    try:
        with open(file_path, 'rb') as f:
            data = pickle.load(f)

        print(f"\n{'='*60}")
        print(f"Verification: {db_name}")
        print(f"{'='*60}")
        print(f"Dataset: {data['dataset']}")
        print(f"Model: {data['model']}")
        print(f"Embedding dimension: {data['embedding_dim']}")
        print(f"Number of proteins: {len(data['vec_dict'])}")

        # Check first protein
        first_prot_id = list(data['vec_dict'].keys())[0]
        first_vec = data['vec_dict'][first_prot_id]
        first_mat = data['mat_dict'][first_prot_id]

        print(f"\nFirst protein: {first_prot_id}")
        print(f"  - Vec shape: {first_vec.shape}")
        print(f"  - Mat shape: {first_mat.shape}")
        print(f"  - Sequence length: {data['length_dict'][first_prot_id]}")

        return data
    except FileNotFoundError:
        print(f"✗ File not found: {file_path}")
        return None

# Verify each dataset
for db_name in ['davis', 'kiba', 'metz']:
    verify_embeddings(db_name)

✗ File not found: ./davis_esmc_pretrain.pkl
✗ File not found: ./kiba_esmc_pretrain.pkl
✗ File not found: ./metz_esmc_pretrain.pkl


In [ ]:
import os

file_names = ['davis_esmc_pretrain.pkl', 'kiba_esmc_pretrain.pkl', 'metz_esmc_pretrain.pkl', 'case_study_esmc_pretrain.pkl']
current_dir = os.getcwd()

print("Địa chỉ cụ thể của các file _esmc_pretrain.pkl:")
for file_name in file_names:
    full_path = os.path.join(current_dir, file_name)
    if os.path.exists(full_path):
        print(f"  - File '{file_name}' TỒN TẠI tại: {full_path}")
    else:
        print(f"  - File '{file_name}' KHÔNG TỒN TẠI tại: {full_path}")

Địa chỉ cụ thể của các file _esmc_pretrain.pkl:
  - File 'davis_esmc_pretrain.pkl' TỒN TẠI tại: /content/Temp/davis_esmc_pretrain.pkl
  - File 'kiba_esmc_pretrain.pkl' TỒN TẠI tại: /content/Temp/kiba_esmc_pretrain.pkl
  - File 'metz_esmc_pretrain.pkl' TỒN TẠI tại: /content/Temp/metz_esmc_pretrain.pkl
  - File 'case_study_esmc_pretrain.pkl' TỒN TẠI tại: /content/Temp/case_study_esmc_pretrain.pkl


In [ ]:
import os
import shutil

file_suffixes = ['davis_esmc_pretrain.pkl', 'kiba_esmc_pretrain.pkl', 'metz_esmc_pretrain.pkl', 'case_study_esmc_pretrain.pkl']
base_source_dir = '/content/Temp'
base_dest_dir = '/content/Temp/data'

print("Di chuyển các file _esmc_pretrain.pkl:")

for file_suffix in file_suffixes:
    dataset_name = file_suffix.split('_')[0] # e.g., 'davis' from 'davis_esmc_pretrain.pkl'
    source_path = os.path.join(base_source_dir, file_suffix)

    # Construct destination directory: /content/Temp/data/davis/
    destination_dataset_dir = os.path.join(base_dest_dir, dataset_name)
    os.makedirs(destination_dataset_dir, exist_ok=True)

    # Construct destination file name: /content/Temp/data/davis/davis_esm_pretrain.pkl
    # Note: The user requested _esm_pretrain.pkl, but files are _esmc_pretrain.pkl.
    # Keeping _esmc_pretrain.pkl for consistency with generated files.
    destination_file_name = f"{dataset_name}_esmc_pretrain.pkl"
    destination_path = os.path.join(destination_dataset_dir, destination_file_name)

    if os.path.exists(source_path):
        try:
            shutil.move(source_path, destination_path)
            print(f"  - Đã di chuyển '{file_suffix}' tới '{destination_path}'")
        except Exception as e:
            print(f"  - Lỗi khi di chuyển '{file_suffix}': {e}")
    else:
        print(f"  - File '{file_suffix}' KHÔNG TỒN TẠI tại '{source_path}', bỏ qua.")

print("Hoàn tất việc di chuyển file.")

# Verify new locations
print("\nKiểm tra lại địa chỉ mới của các file:")
for file_suffix in file_suffixes:
    dataset_name = file_suffix.split('_')[0]
    destination_file_name = f"{dataset_name}_esmc_pretrain.pkl"
    destination_path = os.path.join(base_dest_dir, dataset_name, destination_file_name)
    if os.path.exists(destination_path):
        print(f"  - File '{destination_file_name}' TỒN TẠI tại: {destination_path}")
    else:
        print(f"  - File '{destination_file_name}' KHÔNG TỒN TẠI tại: {destination_path}")

Di chuyển các file _esmc_pretrain.pkl:
  - Đã di chuyển 'davis_esmc_pretrain.pkl' tới '/content/Temp/data/davis/davis_esmc_pretrain.pkl'
  - Đã di chuyển 'kiba_esmc_pretrain.pkl' tới '/content/Temp/data/kiba/kiba_esmc_pretrain.pkl'
  - Đã di chuyển 'metz_esmc_pretrain.pkl' tới '/content/Temp/data/metz/metz_esmc_pretrain.pkl'
  - Đã di chuyển 'case_study_esmc_pretrain.pkl' tới '/content/Temp/data/case/case_esmc_pretrain.pkl'
Hoàn tất việc di chuyển file.

Kiểm tra lại địa chỉ mới của các file:
  - File 'davis_esmc_pretrain.pkl' TỒN TẠI tại: /content/Temp/data/davis/davis_esmc_pretrain.pkl
  - File 'kiba_esmc_pretrain.pkl' TỒN TẠI tại: /content/Temp/data/kiba/kiba_esmc_pretrain.pkl
  - File 'metz_esmc_pretrain.pkl' TỒN TẠI tại: /content/Temp/data/metz/metz_esmc_pretrain.pkl
  - File 'case_esmc_pretrain.pkl' TỒN TẠI tại: /content/Temp/data/case/case_esmc_pretrain.pkl


In [ ]:
!pwd

/content/Temp


In [ ]:
!python code/train.py --fold 0 --cuda "0" --dataset davis --running_set warm --epochs 1 --batch_size 256

Training Fold 0/4
Dataset: davis-warm
Device: cuda (CUDA_VISIBLE_DEVICES=0)
Pretrain-./data/davis/davis_drug_pretrain.pkl
Pretrain-./data/davis/davis_esmc_pretrain.pkl
Loading fold 0 data...
  Train: ./data/dta-5fold-dataset/davis/warm/fold_0_train.csv
  Valid: ./data/dta-5fold-dataset/davis/warm/fold_0_valid.csv
  Test:  ./data/dta-5fold-dataset/davis/warm/fold_0_test.csv
Dataset loaded: 19236 train, 4809 valid, 6011 test samples
/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/content/Temp/code/MyDataset.py:68: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  drug_id, prot_id, label = pair[-3], pair[-2], pair[-1]
Traing Log

In [ ]:
!python regenerate_esmc_embeddings.py --dataset davis --model esmc_300m

Regenerating ESM-C embeddings for DAVIS
Model: esmc_300m (dim=960)
Device: cuda

Loading esmc_300m...
Fetching 4 files: 100% 4/4 [00:00<00:00, 5466.67it/s]
Model loaded!

Loading proteins from: ./data/dta-5fold-dataset/davis/davis_prots.csv
Found 442 unique proteins

Extracting embeddings...
100% 442/442 [02:23<00:00,  3.08it/s]

✅ Successfully saved to: ./data/davis/davis_esmc_pretrain.pkl
Total proteins: 442
Embedding dimension: 960

Verification:
  First protein: AAK1
  Vec shape: (963, 960) ❌
  Mat shape: (1, 963, 960) ❌



In [ ]:
!python code/train.py --fold 0 --cuda "0" --dataset davis --running_set warm --epochs 1 --batch_size 256

Training Fold 0/4
Dataset: davis-warm
Device: cuda (CUDA_VISIBLE_DEVICES=0)
Pretrain-./data/davis/davis_drug_pretrain.pkl
Pretrain-./data/davis/davis_esmc_pretrain.pkl
Loading fold 0 data...
  Train: ./data/dta-5fold-dataset/davis/warm/fold_0_train.csv
  Valid: ./data/dta-5fold-dataset/davis/warm/fold_0_valid.csv
  Test:  ./data/dta-5fold-dataset/davis/warm/fold_0_test.csv
Dataset loaded: 19236 train, 4809 valid, 6011 test samples
/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/content/Temp/code/MyDataset.py:68: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  drug_id, prot_id, label = pair[-3], pair[-2], pair[-1]
Traing Log

In [ ]:
!python code/train.py --fold 0 --cuda "0" --dataset kiba --running_set warm --epochs 1 --batch_size 256

Training Fold 0/4
Dataset: kiba-warm
Device: cuda (CUDA_VISIBLE_DEVICES=0)
Pretrain-./data/kiba/kiba_drug_pretrain.pkl
Pretrain-./data/kiba/kiba_esmc_pretrain.pkl
Loading fold 0 data...
  Train: ./data/dta-5fold-dataset/kiba/warm/fold_0_train.csv
  Valid: ./data/dta-5fold-dataset/kiba/warm/fold_0_valid.csv
  Test:  ./data/dta-5fold-dataset/kiba/warm/fold_0_test.csv
Dataset loaded: 75683 train, 18921 valid, 23650 test samples
/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/content/Temp/code/MyDataset.py:68: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  drug_id, prot_id, label = pair[-3], pair[-2], pair[-1]
Traing Log at fo